In [35]:
import os

import numpy as np

from PIL import Image
from numpy import asarray
from os import listdir

import warnings
warnings.filterwarnings("ignore")

In [36]:
folder_dir = "chest_xray/test/NORMAL/"
data = []
image_size = []
for images in os.listdir(folder_dir):
    image = Image.open(folder_dir + images)
    # convert image to numpy array
    data_i = asarray(image)
    feature_i = data_i.flatten()
    data.append(feature_i); image_size.append(image.size)
data = np.array(data)

In [34]:
image_size

[(1949, 1632),
 (2720, 2458),
 (1340, 917),
 (2366, 2152),
 (1488, 1168),
 (1442, 770),
 (2043, 1217),
 (1816, 1157),
 (1168, 824),
 (1873, 1478),
 (2001, 1725),
 (1936, 1554),
 (1856, 1418),
 (2431, 2195),
 (1778, 1168),
 (2522, 2124),
 (1898, 1403),
 (1304, 952),
 (1784, 1440),
 (1592, 921),
 (1852, 1443),
 (1890, 1334),
 (1322, 650),
 (1572, 1125),
 (1400, 882),
 (1124, 716),
 (1620, 1110),
 (2043, 2087),
 (1434, 906),
 (1720, 1568),
 (2498, 2057),
 (1348, 812),
 (1248, 880),
 (2144, 1916),
 (1784, 1389),
 (1816, 1435),
 (1741, 1504),
 (1522, 1183),
 (2251, 1828),
 (1754, 1187),
 (1616, 1041),
 (1802, 1226),
 (1646, 1214),
 (2022, 1384),
 (1762, 1249),
 (2010, 1705),
 (1786, 1068),
 (1472, 921),
 (1504, 1056),
 (1742, 1392),
 (1827, 1375),
 (1754, 1318),
 (1456, 839),
 (1408, 1040),
 (1550, 1033),
 (2426, 2076),
 (2143, 1576),
 (1376, 1040),
 (2474, 2224),
 (1538, 1202),
 (1762, 1224),
 (1558, 925),
 (1300, 821),
 (1733, 1044),
 (1720, 1303),
 (1560, 1016),
 (2028, 1742),
 (1266, 10